![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2F08+-+R%2Fdepricated&dt=08b+Script+-+Vertex+AI+Custom+Model+-+R+-+Notebook+to+Script.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/08%20-%20R/depricated/08b%20Script%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20Notebook%20to%20Script.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A//raw.githubusercontent.com/statmike/vertex-ai-mlops/main/08%20-%20R/depricated/08b%20Script%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20Notebook%20to%20Script.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/08%20-%20R/depricated/08b%20Script%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20Notebook%20to%20Script.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https%3A//raw.githubusercontent.com/statmike/vertex-ai-mlops/main/08%20-%20R/depricated/08b%20Script%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20Notebook%20to%20Script.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# 08b Script - Vertex AI Custom Model - R - From Notebook to Script

**IN ACTIVE DEVELOPMENT - NOT COMPLETE**
    
Dev Notes:
- Python Kernel:
    - use notebook instance with R kernel
    - or, install R here with needed libraries
    
Workflow:
- Construct R script for creating model with annotations - based on code in `08`
- Test R script at command line
   
**Resources:**
- Notebooks:
    - [08 - Vertex AI Custom Model - R - in Notebook](./08%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20in%20Notebook.ipynb)
   
**Prerequisites:**
- [01 - BigQuery - Table Data Source](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)



---
## Setup

inputs:

In [37]:
SCRIPT_NAME = 'train.R'

In [38]:
SCRIPT_PATH = './code/' + SCRIPT_NAME

packages:

In [39]:
import os
from IPython.display import Markdown as md

environment:

In [40]:
if os.path.exists('code'):
    print('The code directory alredy exists')
else:
    print('Creating the code directory')
    os.makedirs('code')

The code directory alredy exists


---
## Build the ML Training Code
Use the cell magic `%%writefile` to create a single ML training code file named `train.R`.

### Import Libraries

In [41]:
%%writefile {SCRIPT_PATH}

# library import
library(bigrquery)
library(dplyr)

Overwriting ./code/train.R


### Parse Command Line Inputs

In [42]:
%%writefile -a {SCRIPT_PATH}

# inputs
args <- commandArgs(trailingOnly = TRUE)
project_id <- args[1]
region <- args[2]
experiment <- args[3]
series <- args[4]
bq_project <- args[5]
bq_dataset <- args[6]
bq_table <- args[7]
var_target <- args[8]
var_omit <- args[9]

Appending to ./code/train.R


### Retrieve Data From BigQuery

In [43]:
%%writefile -a {SCRIPT_PATH}

# data source
get_data <- function(s){
    query = sprintf('SELECT * EXCEPT(%s, splits) FROM `%s.%s.%s` WHERE splits = "%s"', var_omit, bq_project, bq_dataset, bq_table, s)
    table <- bq_project_query(bq_project, query)
    ds <- bq_table_download(table)
    return(ds)
}
train <- get_data("TRAIN")
test <- get_data("TEST")

Appending to ./code/train.R


### Fit Model

In [44]:
%%writefile -a {SCRIPT_PATH}

# logistic regression model
model <- glm(
    Class ~ .,
    data = train,
    family = binomial)

Appending to ./code/train.R


### Predictions for Test Split

In [45]:
%%writefile -a {SCRIPT_PATH}

# predictions for evaluation
preds <- predict(model, test, type = "response")

Appending to ./code/train.R


### Evaluate The Model

In [46]:
%%writefile -a {SCRIPT_PATH}

# evaluate
actual <- test[, var_target]
names(actual) <- 'actual'
pred <- tibble(round(preds))
names(pred) <- 'pred'
results <- cbind(actual, pred)
cm <- table(results)

Appending to ./code/train.R


### Save The Model

In [47]:
%%writefile -a {SCRIPT_PATH}

# save model to file
saveRDS(model, "model.rds")

path <- sub('gs://', '/gcs/', Sys.getenv('AIP_MODEL_DIR'))
#saveRDS(
#    model,
#    sprintf('%smodel.rds', path)
#)

# use Vertex AI Training Pre-Defined Environment Variables to Write to GCS
#system2('gsutil', c('cp', 'model.rds', Sys.getenv('AIP_MODEL_DIR')))
system2('cp', c('model.rds', path))

Appending to ./code/train.R


---
## Review The Training Code
Read in the completed script and print it out for review below:

In [48]:
with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```r\n\n{data}\n```")

```r


# library import
library(bigrquery)
library(dplyr)

# inputs
args <- commandArgs(trailingOnly = TRUE)
project_id <- args[1]
region <- args[2]
experiment <- args[3]
series <- args[4]
bq_project <- args[5]
bq_dataset <- args[6]
bq_table <- args[7]
var_target <- args[8]
var_omit <- args[9]

# data source
get_data <- function(s){
    query = sprintf('SELECT * EXCEPT(%s, splits) FROM `%s.%s.%s` WHERE splits = "%s"', var_omit, bq_project, bq_dataset, bq_table, s)
    table <- bq_project_query(bq_project, query)
    ds <- bq_table_download(table)
    return(ds)
}
train <- get_data("TRAIN")
test <- get_data("TEST")

# logistic regression model
model <- glm(
    Class ~ .,
    data = train,
    family = binomial)

# predictions for evaluation
preds <- predict(model, test, type = "response")

# evaluate
actual <- test[, var_target]
names(actual) <- 'actual'
pred <- tibble(round(preds))
names(pred) <- 'pred'
results <- cbind(actual, pred)
cm <- table(results)

# save model to file
saveRDS(model, "model.rds")

path <- sub('gs://', '/gcs/', Sys.getenv('AIP_MODEL_DIR'))
#saveRDS(
#    model,
#    sprintf('%smodel.rds', path)
#)

# use Vertex AI Training Pre-Defined Environment Variables to Write to GCS
#system2('gsutil', c('cp', 'model.rds', Sys.getenv('AIP_MODEL_DIR')))
system2('cp', c('model.rds', path))

```